In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
import RPi.GPIO as GPIO
import threading

class WebcamVideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        threading.Thread(target=self.update, args=()).start()
        return self

    def update(self):
        while True:
            if self.stopped:
                return
            (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True

red_pin = 17
green_pin = 18
blue_pin = 27
trig_pin = 23
echo_pin = 24
in1_pin = 19
in2_pin = 20
in3_pin = 21
in4_pin = 22
ena_pin = 5
enb_pin = 6

# GPIO Setup
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
GPIO.setup(red_pin, GPIO.OUT)
GPIO.setup(green_pin, GPIO.OUT)
GPIO.setup(blue_pin, GPIO.OUT)
GPIO.setup(trig_pin, GPIO.OUT)
GPIO.setup(echo_pin, GPIO.IN)
GPIO.setup(in1_pin, GPIO.OUT)
GPIO.setup(in2_pin, GPIO.OUT)
GPIO.setup(in3_pin, GPIO.OUT)
GPIO.setup(in4_pin, GPIO.OUT)
GPIO.setup(ena_pin, GPIO.OUT)
GPIO.setup(enb_pin, GPIO.OUT)

def set_led_color(red, green, blue):
    GPIO.output(red_pin, red)
    GPIO.output(green_pin, green)
    GPIO.output(blue_pin, blue)

def set_led_color_by_result(result):
    if result == 'fire':
        set_led_color(1, 0, 0)
    elif result == 'human':
        set_led_color(0, 1, 0)
    elif result == 'water':
        set_led_color(0, 0, 1)
    else:
        set_led_color(1, 1, 1)

model_save_path = '/home/jemin/Downloads/model_trained_data.h5'
model = load_model(model_save_path)

def control_motor(in1_state, in2_state, in3_state, in4_state, ena_duty_cycle, enb_duty_cycle):
    GPIO.output(in1_pin, in1_state)
    GPIO.output(in2_pin, in2_state)
    GPIO.output(in3_pin, in3_state)
    GPIO.output(in4_pin, in4_state)
    pwm_ena.ChangeDutyCycle(ena_duty_cycle)
    pwm_enb.ChangeDutyCycle(enb_duty_cycle)

pwm_frequency = 1000
pwm_ena = GPIO.PWM(ena_pin, pwm_frequency)
pwm_enb = GPIO.PWM(enb_pin, pwm_frequency)
pwm_ena.start(0)
pwm_enb.start(0)

def measure_distance():
    GPIO.output(trig_pin, GPIO.LOW)
    time.sleep(0.5)
    GPIO.output(trig_pin, GPIO.HIGH)
    time.sleep(0.00001)
    GPIO.output(trig_pin, GPIO.LOW)
    while GPIO.input(echo_pin) == 0:
        pulse_start = time.time()
    while GPIO.input(echo_pin) == 1:
        pulse_end = time.time()
    pulse_duration = pulse_end - pulse_start
    distance = pulse_duration * 17150
    distance = round(distance, 2)
    return distance

cap = WebcamVideoStream(src=0).start()

while True:
    frame = cap.read()
    if frame is not None:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        gray = cv2.resize(gray, (128, 128))
        gray = gray / 255.0
        input_img = gray.reshape(1, 128, 128, 1)

        prediction = model.predict(input_img)
        probability = np.amax(prediction)
        class_index = np.argmax(prediction)

        if class_index == 0:
            label = 'fire'
        elif class_index == 1:
            label = 'human'
        elif class_index == 2:
            label = 'water'
        else:
            continue

        distance = measure_distance()
        print("Distance:", distance, "cm")

        if probability >= 0.97:
            text = f'{label} ({probability * 100:.2f}%)'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            set_led_color_by_result(label)

            if distance >= 14:
                control_motor(1, 0, 1, 0, 65, 65)
                time.sleep(1)
                control_motor(0, 0, 0, 0, 0, 0)
                time.sleep(1)
            else:
                control_motor(0, 1, 1, 0, 65, 95)
        else:
            text = 'Low probability'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            set_led_color_by_result('')

            if distance >= 14:
                control_motor(1, 0, 1, 0, 65, 65)
            else:
                control_motor(0, 1, 1, 0, 65, 95)

        cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.stop()
cv2.destroyAllWindows()
GPIO.cleanup()